# Imports

In [198]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance
from scipy.spatial.distance import cdist


# Input

In [199]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [200]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [201]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [202]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [203]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[1.24307132 1.24379455 1.24307132 ... 1.15725753 1.67985361 1.15725753]
[0.94522671 0.94577665 0.94522671 ... 0.87997424 1.27735431 0.87997424]
[0.73842696 0.73854357 0.73842696 ... 0.72435442 0.80257508 0.72435442]
(5999,)


### Multinomial Sampling

In [204]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 0 0 0]


In [205]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(graph)[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[86.45986  ,  5.7970243, 91.5185   ],
       [84.7046   ,  5.7970243, 93.18418  ],
       [83.39875  ,  5.7970243, 94.13294  ],
       [87.93294  ,  5.7970243, 89.59875  ],
       [88.354774 ,  5.7970243, 88.910385 ]], dtype=float32)

# KNN extended graph

In [206]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [207]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for poly in indices:
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11738


# Edge Predictor

In [208]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [209]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [210]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [211]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [212]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 6)	1
  (1, 7)	1
  (1, 10)	1
  (1, 11)	1
  (1, 12)	1
  (1, 13)	1
  (1, 22)	1
  :	:
  (1498, 613)	1
  (1498, 614)	1
  (1498, 615)	1
  (1498, 616)	1
  (1498, 618)	1
  (1498, 1112)	1
  (1498, 1350)	1
  (1498, 1461)	1
  (1498, 1476)	1
  (1498, 1499)	1
  (1499, 608)	1
  (1499, 609)	1
  (1499, 610)	1
  (1499, 611)	1
  (1499, 612)	1
  (1499, 613)	1
  (1499, 614)	1
  (1499, 615)	1
  (1499, 616)	1
  (1499, 617)	1
  (1499, 1112)	1
  (1499, 1350)	1
  (1499, 1461)	1
  (1499, 1476)	1
  (1499, 1498)	1
[[0.0718114  0.07181526 0.07181526 ... 0.07199187 0.07171339 0.07172243]
 [1.08182918 0.0663841  0.0663841  ... 0.06623425 0.06646453 0.06645757]
 [1.01396688 0.56197368 0.06179888 ... 0.06146122 0.06198864 0.06197196]
 ...
 [1.0871311  1.18150864 1.2711361  ... 0.06563128 0.0641331  0.06418331]
 [

In [213]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [214]:
print(A_s)  # symmétrique
print(A_s.shape)

[[1.15060574e-19 1.05676772e-19 9.81622640e-20 ... 1.21834869e-18
  8.75277257e-19 8.92162484e-19]
 [1.05676772e-19 9.71655450e-20 9.07802782e-20 ... 1.13025777e-18
  8.21732058e-19 8.37318937e-19]
 [9.81622640e-20 9.07802782e-20 8.48410334e-20 ... 1.05573115e-18
  7.72362935e-19 7.86856508e-19]
 ...
 [1.21834869e-18 1.13025777e-18 1.05573115e-18 ... 2.00559219e-17
  1.60032062e-17 1.64091099e-17]
 [8.75277257e-19 8.21732058e-19 7.72362935e-19 ... 1.60032062e-17
  1.39940651e-17 1.43340385e-17]
 [8.92162484e-19 8.37318937e-19 7.86856508e-19 ... 1.64091099e-17
  1.43340385e-17 1.46895447e-17]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [215]:
nodes = list(extended_graph.nodes())
edges = list(extended_graph.edges())

igraph_g = igraphGraph(directed=extended_graph.is_directed())

igraph_g.add_vertices(nodes)
igraph_g.add_edges([(nodes.index(u), nodes.index(v)) for u, v in edges])
print(igraph_g.summary())

IGRAPH UN-- 1500 11738 -- 
+ attr: name (v)


In [216]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[ 932 1169 1208]
 [1169 1207 1208]
 [ 915  916  919]]
32379


In [217]:
triangles = np.array(igraph_g.vs['name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[76.92624   -6.5098743  1.6606928]
  [75.48362   -5.0095077  2.311571 ]
  [75.60344   -4.4479885  2.754807 ]]

 [[75.48362   -5.0095077  2.311571 ]
  [75.49088   -4.4479885  2.6974556]
  [75.60344   -4.4479885  2.754807 ]]

 [[75.35673   -2.6241417  3.6182384]
  [75.12372   -2.6241417  3.419231 ]
  [75.39199   -3.2518802  3.3159306]]]
32379


In [218]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(igraph_g.vs))
p_init[i] = (A_s_ij + A_s_ik + A_s_jk) / 3

#### Calculate barycenter

In [219]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(32379, 3)
[[76.00443   -5.322457   2.242357 ]
 [75.52598   -4.635162   2.5879445]
 [75.29081   -2.833388   3.4511335]]


#### KNN Tri

In [220]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [221]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(32379, 19, 3)


#### calculate e norm matrix

In [222]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(32379, 3)
[[2.180801  2.682957  0.7253404]
 [0.6813689 0.7253404 0.1263255]]


#### Calculate r

In [223]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(32379,)

In [224]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(32379, 19)
(32379, 19)


In [225]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(32379, 19, 5)


#### Calculate f

In [226]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.9999999999999984
(1500,)
[0.00066589 0.00066589 0.00066589 ... 0.00066589 0.00066589 0.00066589]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_5324\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [227]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[:1])

(500, 3, 3)
[[[81.80516   8.722893 84.667404]
  [81.9773    8.722893 84.306496]
  [82.130325  8.722893 83.93707 ]]]


In [228]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 506
Number of edges: 996


In [229]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [230]:
keys_input = np.array(graph)
print("Les points avant K-NN : ", keys_input)
print("dim points avant K-NN : ", len(keys_input))  #return 5999

Les points avant K-NN :  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]
dim points avant K-NN :  5999


Récupération des points de l'espace de départ Ps

In [231]:
keys_output = np.array(extended_graph)
print("sampled points : ", keys_output)
print("dim sample points : ", len(keys_output))  #return: 50

sampled points :  [[86.45986    5.7970243 91.5185   ]
 [86.11996    6.2811418 91.212456 ]
 [86.62873    6.2811418 90.61676  ]
 ...
 [83.96025   -7.5075808 91.15137  ]
 [75.848564   3.8609922 81.475716 ]
 [75.887344   3.8609922 81.29327  ]]
dim sample points :  1500


Distance Matrix

In [254]:
def matrix_distance_euclidienne_barycentres(barycentre_b, barycentre_b_hat):
    """
    Calculate distances between barycenters b et b_hat after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """
    return cdist(barycentre_b, barycentre_b_hat, metric='sqeuclidean').T

distances_matrix = matrix_distance_euclidienne_barycentres(keys_input, keys_output)
print("Squared Euclidean Distances Matrix Shape:", distances_matrix.shape)
print("Distances Matrix:\n", distances_matrix)




Squared Euclidean Distances Matrix Shape: (1500, 5999)
Distances Matrix:
 [[ 7867.05161297  5768.45601571  7859.46939709 ... 11166.15440199
  12036.37464803 11041.3379659 ]
 [ 7812.28735261  5721.41934522  7804.0436133  ... 11096.11569302
  11958.69991718 10971.8209499 ]
 [ 7720.04808769  5644.56023521  7711.82999046 ... 11015.47838291
  11891.0524893  10891.96295073]
 ...
 [ 7756.91011183  5672.15327213  7766.91122206 ... 10717.55004298
  11524.925984   10589.87996349]
 [ 6054.6841657   4215.60208673  6049.14412899 ...  8596.73759244
   9224.93989876  8484.57435921]
 [ 6026.4390516   4192.06502039  6020.9009694  ...  8566.42177451
   9195.79349292  8454.49722827]]


In [255]:
# Calculer le min des x de distances_matrix
def min_pour_x_list(distances_matrix, x_in):
    return np.min(distances_matrix[:, x_in])

# Calculer le min des y de distances_matrix
def min_pour_y_list(distances_matrix, y_out):
    return np.min(distances_matrix[y_out, :])


In [256]:
def d_P_Ps(keys_x, keys_y, p_y):
    """
    Calculate the Probabilistic Chamfer distance between two sets of points.

    Parameters:
    - keys_x (list): The input vertex set, a list of tuples.
    - keys_y (list): The sampled points, a list of tuples.
    - p_y (list): Their respective probabilities, a weight associated with each point in keys_y.

    Returns:
        chamfer_distance (float): The Probabilistic Chamfer distance between the two sets of points.
    """
    min_for_x = [p_y[i] * min_pour_x_list(distances_matrix, i) for i in range(len(keys_x))]
    min_for_y = [p_y[j] * min_pour_y_list(distances_matrix, j) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(min_for_y)
    sum_for_x = np.sum(min_for_x)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance


In [257]:
print("PROBA : ", normalized_inclusion_score)

PROBA :  [0.00021886 0.0002189  0.00021886 ... 0.00021469 0.00023788 0.00021469]


In [258]:
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Chamfer Distance:", prob_chamfer_dist)

Chamfer Distance: 3.5601460037725214


## Probabilistic Surfaces Distance

barycentres b et b_hat

In [259]:
# Creation igraph
nodes_original = list(graph.nodes())
edges_original = list(graph.edges())
igraph_g_original = igraphGraph(directed=extended_graph.is_directed())
igraph_g_original.add_vertices(nodes_original)
igraph_g_original.add_edges([(nodes_original.index(u), nodes_original.index(v)) for u, v in edges_original])
print(igraph_g_original.summary())

#Find triangles
triangles_ids_igraph_original = np.array(igraph_g_original.cliques(min=3, max=3))
triangles = np.array(igraph_g_original.vs['name'])[triangles_ids_igraph_original]
print("triangles shape : ", triangles.shape)
print(triangles[0])

# Calculate barycenters
b = np.mean(triangles, axis=1)
print("shape b : ", b.shape)
print("Barycentre b: ", b)

IGRAPH UN-- 5999 17991 -- 
+ attr: name (v)
triangles shape :  (11994, 3, 3)
[[39.        8.793697 -4.666995]
 [51.        8.793697 -4.666995]
 [39.        8.793697 -5.333005]]
shape b :  (11994, 3)
Barycentre b:  [[43.          8.793697   -4.8889985 ]
 [55.          1.0515273   3.7313068 ]
 [55.         -8.515905   -2.804371  ]
 ...
 [75.05613    -0.94137865 21.        ]
 [75.01313    -0.3882971   8.099999  ]
 [75.03516    -0.72036856 25.300001  ]]


In [260]:
b_hat = barycenters
print(b_hat.shape)
print("B_HAT : ", b_hat)

(32379, 3)
B_HAT :  [[76.00443    -5.322457    2.242357  ]
 [75.52598    -4.635162    2.5879445 ]
 [75.29081    -2.833388    3.4511335 ]
 ...
 [75.177986   -2.195629   81.402336  ]
 [75.06988    -1.0990705  81.337135  ]
 [75.058784   -0.21676879 81.2705    ]]


test matrice des normes euclidiennes entre b et b_hat

In [261]:
distances = matrix_distance_euclidienne_barycentres(b, b_hat)
print(distances.shape)
print("Distances euclidiennes entre les barycentres b et b_hat :\n", distances)


(32379, 11994)
Distances euclidiennes entre les barycentres b et b_hat :
 [[1339.41461725  484.03083686  476.85377059 ...  371.94229538
    59.64058746  553.77366603]
 [1294.17818779  454.96148715  465.45301444 ...  352.8685797
    48.68162275  531.40402943]
 [1247.44330439  426.88801987  483.13929615 ...  311.59749029
    27.66753028  481.90322797]
 ...
 [8602.38251559 6450.48394496 7537.86656395 ... 3650.03020151
  5376.52619024 3149.66878316]
 [8561.29006183 6430.08967965 7537.60253167 ... 3640.59495376
  5364.1864921  3140.30502681]
 [8532.41337755 6416.28992173 7539.8144876  ... 3633.05825883
  5353.95368706 3132.95093367]]


matrice sqeuclidean b

In [268]:
def normes_euclidiennes_carrees_b(bar_b):
    # Calculer la matrice des normes euclidiennes au carré
    normes_carrees = np.sum(bar_b**2, axis=1)

    return normes_carrees

matrix_b = normes_euclidiennes_carrees_b(b)
print(matrix_b.shape)
print(matrix_b)

(11994,)
[1950.2314 3040.0283 3105.3853 ... 6075.3086 5692.7305 6270.884 ]


In [265]:
def matrix_distances_after_knn(b_hat_n, number_neigh=3):
    """
    Calculate distances between barycenters after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """

    # Connect the extended graph using the k-NN process
    extended_graph = knn_and_extended_graph(b_hat_n, number_neigh)

    # Extract barycenters after the k-NN process
    b_hat_extended = np.array(list(extended_graph._node.keys()))

    # Calculate distances after k-NN
    distances_after = cdist(b_hat_extended, b_hat_extended)

    return distances_after

# Example usage with b_hat
distances_after = matrix_distances_after_knn(b_hat)

# Print the results
print("Distances After k-NN:\n", distances_after)

Distances After k-NN:
 [[ 0.          0.10401448  0.10536496 ... 79.23180989 79.21296825
  79.19854616]
 [ 0.10401448  0.          0.15702782 ... 79.14440044 79.12624607
  79.11235943]
 [ 0.10536496  0.15702782  0.         ... 79.20928355 79.19032019
  79.17589764]
 ...
 [79.23180989 79.14440044 79.20928355 ...  0.          1.10377604
   1.98698228]
 [79.21296825 79.12624607 79.19032019 ...  1.10377604  0.
   0.884884  ]
 [79.19854616 79.11235943 79.17589764 ...  1.98698228  0.884884
   0.        ]]


In [182]:
def probabilistic_surface_distance_extended(b_i, b_hat_i, p_b_hat_i):
    """
    Calculate Probabilistic Surfaces Distance 

    Parameters:
    - b: array-like, shape (N, 3)
        Barycenters of the ground truth surface.
    - b_hat: array-like, shape (M, 3)
        Barycenters of the generated triangles.
    - p_b_hat: array-like, shape (M,)
        Probabilities corresponding to the generated triangles.

    Returns:
    - d_S_Ss: float
        Probabilistic Surfaces Distance.
    """
    # Debugging print statements
    print("b shape:", b_i.shape)
    print("b_hat shape:", b_hat_i.shape)
    print("p_b_hat shape:", p_b_hat_i.shape)

    p_b_hat_i = normalized_inclusion_score.reshape((-1, 1))
    dist_matrixs_1= distances_after - matrix_b
    #dist_matrixs_2= matrix_b - distances_after

    d_S_Ss = np.sum(p_b_hat_i * np.min(np.sum(dist_matrixs_1)**2))

    

    return d_S_Ss

result_distance = probabilistic_surface_distance_extended(b, b_hat, normalized_inclusion_score)
print("Probabilistic Surfaces Distance:", result_distance)

b shape: (5999, 3)
b_hat shape: (1500, 3)
p_b_hat shape: (5999,)


ValueError: operands could not be broadcast together with shapes (1500,1500) (5999,) 

## Triangle Collision Loss

In [ ]:
def triangle_collision_loss(generated_triangles, probabilities):
    """
    Calculate triangle collision loss to penalize triangles that penetrate each other.

    Parameters:
    - generated_triangles (numpy.ndarray): Array representing the generated triangles.
    - probabilities (numpy.ndarray): Array of probabilities for each generated triangle.

    Returns:
    - float: Triangle collision loss.
    """
    num_triangles = len(generated_triangles)

    # Create a 3D array to represent all combinations of triangle pairs
    triangle_pairs_i, triangle_pairs_j = np.meshgrid(range(num_triangles), range(num_triangles), indexing='ij')

    # Filter out pairs where i is not equal to j
    valid_pairs_mask = triangle_pairs_i != triangle_pairs_j

    # Extract triangles and probabilities for each pair
    triangles_i = generated_triangles[triangle_pairs_i[valid_pairs_mask]]
    triangles_j = generated_triangles[triangle_pairs_j[valid_pairs_mask]]
    probs_i = probabilities[triangle_pairs_i[valid_pairs_mask]]

    # Calculate the plane normals and distances for all triangles
    plane_normals = np.cross(triangles_i[:, 1] - triangles_i[:, 0], triangles_i[:, 2] - triangles_i[:, 0])

    # Calculate the edge vectors for all edges of triangles_j
    edge_vectors = triangles_j[:, [1, 2, 0], :] - triangles_j[:, [0, 1, 2], :]

    # Calculate the dot product of edge vectors and plane normals
    dot_products = np.sum(np.cross(edge_vectors, plane_normals[:, np.newaxis, :]) * triangles_i[:, 0], axis=2)

    # Count the number of faces penetrated for each triangle
    faces_penetrated = np.sum(dot_products < 0, axis=1)

    # Calculate the loss for each triangle
    loss_per_triangle = probs_i * faces_penetrated

    # Sum up the losses and normalize by the number of triangles
    triangle_losses = np.sum(loss_per_triangle) / num_triangles

    return triangle_losses

